In [60]:
import numpy as np 
import matplotlib.pyplot as plt
import stumpy
import pandas as pd
from scipy import signal
from sklearn.linear_model import Ridge
from scipy.spatial import distance
from fatf.utils.kernels import exponential_kernel 

import warnings
warnings.filterwarnings('ignore')

import importlib
import sys
sys.path.append('../')
#from Utils.explanationsnew import LIMESegment, NEVES, LEFTIST, NNSegment, RBP, background_perturb
from Utils.data import loadUCRDataID
from Utils.models import *
from Utils.metrics import *

In [2]:
dataset_map = [('Coffee', 0),
                ('Strawberry', 1),
                   ('GunPointOldVersusYoung', 2),
                   ('HandOutlines', 3),
                    ('yoga', 4),
                    ('ECG200', 5),
                    ('GunPointMaleVersusFemale', 6),
                    ('DodgerLoopGame', 7),
                    ('Chinatown', 8),
                    ('FreezerSmallTrain', 9),
                    ('HouseTwenty', 10),
                    ('WormsTwoClass', 11)
                    ]

In [3]:
datasets = {}
for data_idx in dataset_map:
    datasets[data_idx[0]] = loadUCRDataID(data_idx[1])

Loading train / test dataset :  ../Data/Coffee_TRAIN ../Data/Coffee_TEST
Loading train / test dataset :  ../Data/Strawberry_TRAIN ../Data/Strawberry_TEST
Loading train / test dataset :  ../Data/GunPointOldVersusYoung_TRAIN ../Data/GunPointOldVersusYoung_TEST
Loading train / test dataset :  ../Data/HandOutlines_TRAIN ../Data/HandOutlines_TEST
Loading train / test dataset :  ../Data/Yoga_TRAIN ../Data/Yoga_TEST
Loading train / test dataset :  ../Data/ECG200_TRAIN ../Data/ECG200_TEST
Loading train / test dataset :  ../Data/GunPointMaleVersusFemale_TRAIN ../Data/GunPointMaleVersusFemale_TEST
Loading train / test dataset :  ../Data/DodgerLoopGame_TRAIN ../Data/DodgerLoopGame_TEST
Loading train / test dataset :  ../Data/Chinatown_TRAIN ../Data/Chinatown_TEST
Loading train / test dataset :  ../Data/FreezerSmallTrain_TRAIN ../Data/FreezerSmallTrain_TEST
Loading train / test dataset :  ../Data/HouseTwenty_TRAIN ../Data/HouseTwenty_TEST
Loading train / test dataset :  ../Data/WormsTwoClass_TRAIN

In [4]:
"""Datasets['Coffee'] : Train x, Train y, Test x, Test y

Train x : N x T x 1
Train y: N x 1 
Test x: N' x T x 1
Test y: N' x 2"""

"Datasets['Coffee'] : Train x, Train y, Test x, Test y\n\nTrain x : N x T x 1\nTrain y: N x 1 \nTest x: N' x T x 1\nTest y: N' x 2"

In [5]:
for data_idx in datasets.keys():
    print(datasets[data_idx][0].shape)

(28, 286, 1)
(613, 235, 1)
(136, 150, 1)
(1000, 2709, 1)
(300, 426, 1)
(100, 96, 1)
(135, 150, 1)
(20, 288, 1)
(20, 24, 1)
(28, 301, 1)
(40, 2000, 1)
(181, 900, 1)


In [6]:
BATCH_SIZES = [8, 64, 32, 64, 64, 16, 16, 4, 4, 4, 8, 16]
WINDOW_SIZES = [20, 20, 10, 100, 50, 10, 10, 20, 3, 10, 200, 100]
CPS = [5, 5, 4, 8, 5, 3, 4, 5, 2, 5, 8, 6]

In [7]:
models = ['knn','cnn','lstmfcn']
trained_models = {}
i = 0
for data_idx in datasets.keys():
    print(data_idx)
    trained_models[data_idx] = {}
    trained_models[data_idx]['knn'] = train_KNN_model(datasets[data_idx][0],datasets[data_idx][1])
    model_cnn = make_CNN_model(datasets[data_idx][0].shape[1:])
    trained_models[data_idx]['cnn'] = train_CNN_model(model_cnn,
                                                      datasets[data_idx][0],
                                                      datasets[data_idx][1],
                                                      epochs=2,
                                                      batch_size=BATCH_SIZES[i])[0]
    model_lstmfcn = make_LSTMFCN_model(datasets[data_idx][0].shape[1])
    trained_models[data_idx]['lstmfcn'] = train_LSTMFCN_model(model_lstmfcn,
                                                      datasets[data_idx][0],
                                                      datasets[data_idx][1],
                                                      datasets[data_idx][2],
                                                      datasets[data_idx][3],
                                                      epochs=2,
                                                      batch_size=BATCH_SIZES[i])
    i = i + 1

Coffee
Epoch 1/2


2024-11-06 21:28:20.610864: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


3/3 [==============================] - 1s 104ms/step - loss: 0.6771 - sparse_categorical_accuracy: 0.6364 - val_loss: 0.6972 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 2/2
3/3 [==============================] - 0s 17ms/step - loss: 0.6364 - sparse_categorical_accuracy: 0.6364 - val_loss: 0.6903 - val_sparse_categorical_accuracy: 1.0000
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 286, 1)]     0           []                               
                                                                                                  
 permute (Permute)              (None, 1, 286)       0           ['input_2[0][0]']                
                                                                                                  
 conv1d_3 (Conv1D)              (None, 1, 

2024-11-06 21:28:21.305585: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-11-06 21:28:21.306119: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-11-06 21:28:21.306789: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/2


...

Epoch 1/2


2024-11-06 21:29:01.613835: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-11-06 21:29:01.614540: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-11-06 21:29:01.615028: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

12/12 - 2s - loss: 0.7969 - accuracy: 0.5801 - val_loss: 0.6996 - val_accuracy: 0.5325 - lr: 0.0010 - 2s/epoch - 145ms/step
Epoch 2/2


2024-11-06 21:29:03.133574: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-11-06 21:29:03.134269: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-11-06 21:29:03.134766: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

12/12 - 1s - loss: 0.4554 - accuracy: 0.7901 - val_loss: 0.6958 - val_accuracy: 0.5584 - lr: 0.0010 - 842ms/epoch - 70ms/step


In [7]:
"""model_KNN shape : N x T 
## model_CNN/LSTMFCN shape  : N x T x 1


SyntaxError: incomplete input (3663073547.py, line 1)

In [8]:
MODEL_TYPES = ['class','proba','proba']

In [23]:
def reshaper(x,j):
    if j == 0:
        return x.reshape(x.shape[0])
    else:
        return x
#print(datasets.keys())
print(reshaper(datasets[data_idx][2][0],0))
#print(reshaper(datasets[data_idx][2][0],0))

[-0.57437159 -0.52338044 -0.44876379 -0.46651461 -0.59004311 -0.73918779
 -0.80091704 -0.81514776 -0.84054255 -0.8663083  -0.87463199 -0.84164884
 -0.82459755 -0.82119474 -0.87320791 -0.95264845 -0.99157859 -0.99606863
 -0.97311994 -0.96290781 -0.9758126  -0.99099643 -1.0034398  -1.0318264
 -1.0288549  -1.0003457  -0.96893924 -0.93857634 -0.94098649 -0.94073583
 -0.95839386 -0.98225027 -0.97491265 -0.97035137 -0.94754631 -0.94660448
 -0.92715433 -0.85785469 -0.84109419 -0.79946847 -0.71206024 -0.66972837
 -0.62611371 -0.53080418 -0.44999816 -0.39837753 -0.37412205 -0.38281253
 -0.37573877 -0.32695258 -0.29110688 -0.29491244 -0.26288307 -0.20024618
 -0.16225478 -0.12686992  0.00417871  0.13997716  0.21599448  0.31917355
  0.32410045  0.29486424  0.32026563  0.30926601  0.29323908  0.35810612
  0.48667162  0.68688947  0.75717585  0.68730592  0.72987891  0.75600485
  0.83824631  0.83549145  0.77260256  0.76120455  0.68590876  0.64448829
  0.61831328  0.63251105  0.68554323  0.75817086  0.

In [57]:
from scipy import signal
import numpy as np
import stumpy
from sklearn.linear_model import Ridge
from fastdtw import fastdtw
import random


def NNSegment(t, window_size, change_points):
    """Return the change points of given time series
    Input: 
        t - numpy array of size T
        window_size - int where window_size < T 
        change_points - user specified number of change points
    Output: 
        np.array of change point indexes 
    """
    """Return the change points of given time series
    Input: 
        t - numpy array of size T
        window_size - int where window_size < T 
        change_points - user specified number of change points
    Output: 
        np.array of change point indexes 
    """
    mp = stumpy.stump(t, m=window_size)
    proposed_cp = [i for i in range(0,mp.shape[0]-1) if mp[i+1,1] != mp[i,1] + 1]
    tolerance = int(window_size/2)
    variances = []
    for idx in proposed_cp:
        mean_change = np.abs(np.mean(t[idx-tolerance:idx]) - np.mean(t[idx:idx+tolerance]))
        std_change = np.abs(np.std(t[idx-tolerance:idx]) - np.std(t[idx:idx+tolerance])) 
        std_mean = np.mean([np.std(t[idx-tolerance:idx]),np.std(t[idx:idx+tolerance])])
        variances.append(mean_change*std_change/std_mean)
    sorted_idx = np.flip(np.array(variances).argsort())
    sorted_cp = [proposed_cp[idx] for idx in sorted_idx]
    selected_cp = []
    covered = list(np.arange(0,tolerance))
    ic, i = 0, 0 
    while ic < change_points:
        if i == len(sorted_cp):
            break
        if sorted_cp[i] not in covered:
            ic += 1
            selected_cp.append(sorted_cp[i])
            covered = covered + list(np.arange(sorted_cp[i],sorted_cp[i]+tolerance))
            covered = covered + list(np.arange(sorted_cp[i]-tolerance,sorted_cp[i]))
        i +=1 
    selected_cp = np.sort(np.asarray(selected_cp))
    return(list(selected_cp))
    
def backgroundIdentification(global_signals,F=40):
    test_sig = global_signals[0].flatten()
    f, t, Zxx = signal.stft(test_sig,1,nperseg = F)
    for i in range(1,len(global_signals)):
        test_sig = global_signals[i].flatten()
        f0, t0, Zxx0 = signal.stft(test_sig,1,nperseg=F)
        assert Zxx.shape == Zxx0.shape, f"Shape mismatch at signal {i}"
        Zxx = Zxx + Zxx0
    frequency_composition_abs = np.abs(Zxx)
    measures = []
    for freq,freq_composition in zip(f,frequency_composition_abs):
        measures.append(np.mean(freq_composition)/np.std(freq_composition))
    max_value = max(measures)
    selected_frequency = measures.index(max_value)
    weights = 1-(measures/sum(measures))
    dummymatrix = np.zeros((len(f),len(t)))
    dummymatrix[selected_frequency,:] = 1  
    #Option to admit information from other frequency bands
    """dummymatrix = np.ones((len(f),len(t)))
    for i in range(0,len(weights)):
        dummymatrix[i,:] = dummymatrix[i,:] * weights[i]"""
    
    background_frequency = Zxx * dummymatrix
    _, xrec = signal.istft(background_frequency, 1)
    signal_template = global_signals[0].reshape(global_signals[0].shape[0])
    xrec = xrec[:signal_template.shape[0]]
    xrec = xrec.reshape(signal_template.shape)
    print(xrec.shape)
    return xrec
    
def RBP(generated_samples_interpretable, original_signal, global_signals, segment_indexes, f):
    original_signal = original_signal.flatten()
    generated_samples_raw = []
    xrec = backgroundIdentification(global_signals)
    for sample_interpretable in generated_samples_interpretable:
        raw_signal = original_signal.copy()
        for index in range(0,len(sample_interpretable)-1):
            if sample_interpretable[index] == 0:
                index0 = segment_indexes[index]
                index1 = segment_indexes[index+1]
                raw_signal[index0:index1] = xrec[index0:index1]
        generated_samples_raw.append(np.asarray(raw_signal))
    return np.asarray(generated_samples_raw)

def RBPIndividual(original_signal, index0, index1):
    xrec = backgroundIdentification(original_signal)
    raw_signal = original_signal.copy()
    raw_signal[index0:index1] = xrec[index0:index1]
    return raw_signal

def LIMESegment(example, datasets, model, model_type='class', distance='dtw', n=100, window_size=None, cp=None, f=None):
    if window_size is None:
        window_size =int(example.shape[0]/5)
    if cp is None:
        cp = 3
    if f is None: 
        f = int(example.shape[0]/10)
    cp_indexes = NNSegment(example.reshape(example.shape[0]), window_size, cp)
    segment_indexes = [0] + cp_indexes + [-1]
    generated_samples_interpretable = [np.random.binomial(1, 0.5, len(cp_indexes)+1) for _ in range(0,n)]
    #
    generated_samples_raw = RBP(generated_samples_interpretable, example, datasets, segment_indexes, f)
    #
    print(f'limesegment sample:{np.asarray(generated_samples_raw).shape}')
    sample_predictions = model.predict(generated_samples_raw)
    if model_type == 'proba':
        y_labels = np.argmax(sample_predictions, axis=1)
    else:
        y_labels = sample_predictions
    
    if distance == 'dtw':
        distances = np.asarray([fastdtw(example, sample)[0] for sample in generated_samples_raw])
        weights = np.exp(-(np.abs((distances - np.mean(distances))/np.std(distances)).reshape(n,)))
    elif distance == 'euclidean':
        distances = np.asarray([np.linalg.norm(np.ones(len(cp_indexes)+1)-x) for x in generated_samples_interpretable])
        weights = np.exp(-(np.abs(distances**2/0.75*(len(segment_indexes)**2)).reshape(n,)))
    clf = Ridge()
    clf.fit(generated_samples_interpretable,y_labels, weights)
    return clf.coef_, segment_indexes

def background_perturb(original_signal, index0, index1, X_background):
    perturbed_signal = original_signal.copy()
    selected_background_ts = X_background[random.randint(0, 20)]
    perturbed_signal[index0:index1] = selected_background_ts.reshape(perturbed_signal.shape)[index0:index1]   
    return np.asarray(perturbed_signal)

def mean_perturb(original_signal, index0, index1, mean_value, ws):
    perturbed_signal = original_signal.copy()
    mean_signal = np.ones(original_signal.shape)*mean_value
    perturbed_signal[index0:index1] = mean_signal[index0:index1]
    return perturbed_signal

def calculate_mean(cp_indexes, X_background, ws):
    sample_averages = []
    for ts in X_background: 
        window_averages = np.mean([np.mean(ts[i:i+ws]) for i in cp_indexes])
        sample_averages.append(window_averages)
    return np.mean(sample_averages)


def LEFTIST(example, model,  X_background, model_type="class", n=100):
    ts_length = example.shape[0]
    cp_indexes = [i for i in range(0,example.shape[0],int(example.shape[0]/10))]
    example_interpretable = np.ones(len(cp_indexes))
    generated_samples_interpretable = [ np.random.binomial(1, 0.5, len(cp_indexes)) for _ in range(0,n)]
    generated_samples_original = []
    segment_indexes = cp_indexes + [-1]
    for sample_interpretable in generated_samples_interpretable:
        raw_sample = example.copy()
        for index in range(0,len(sample_interpretable)):
            if sample_interpretable[index] == 0:
                index0 = segment_indexes[index]
                index1 = segment_indexes[index+1]
                raw_sample = background_perturb(raw_sample,index0,index1,X_background)
        generated_samples_original.append((raw_sample))

    samples_2d = np.asarray(generated_samples_original)
    samples_2d = samples_2d.reshape(samples_2d.shape[0], samples_2d.shape[1])
    print(f'leftist sample:{np.asarray(samples_2d).shape}')
    sample_predictions = model.predict(np.asarray(samples_2d))
    if model_type == 'proba':
        y_labels = np.argmax(sample_predictions, axis=1)
    else:
        y_labels = sample_predictions

    distances = np.asarray([np.linalg.norm(np.ones(len(cp_indexes))-x) for x in generated_samples_interpretable])
    weights = np.exp(-(np.abs(distances**2/(len(segment_indexes)**2))))
    clf = Ridge()
    clf.fit(generated_samples_interpretable,y_labels,weights)
    return clf.coef_, cp_indexes

    distances = np.asarray([np.linalg.norm(np.ones(len(cp_indexes))-x) for x in generated_samples_interpretable])
    weights = np.exp(-(np.abs(distances**2/(len(segment_indexes)**2))))
    clf = Ridge()
    clf.fit(generated_samples_interpretable,y_labels,weights)
    return clf.coef_, cp_indexes

def NEVES(example, model, X_background, model_type="class", n=100):
    print(example.shape)
    ts_length = example.shape[0]
    cp_indexes = [i for i in range(0,example.shape[0],int(example.shape[0]/10))]
    example_interpretable = np.ones(len(cp_indexes))
    generated_samples_interpretable = [ np.random.binomial(1, 0.5, len(cp_indexes)) for _ in range(0,n)]
    generated_samples_original = []
    mean_perturb_value = calculate_mean(cp_indexes, X_background, int(cp_indexes[1]-cp_indexes[0]))
    segment_indexes = cp_indexes + [ts_length]
    for sample_interpretable in generated_samples_interpretable:
        raw_sample = example.copy()
        for index in range(0,len(sample_interpretable)):
            if sample_interpretable[index] == 0:
                index0 = segment_indexes[index]
                index1 = segment_indexes[index+1]
                raw_sample = mean_perturb(raw_sample,index0,index1,mean_perturb_value,int(cp_indexes[1]-cp_indexes[0]))
        generated_samples_original.append(raw_sample)
    
    samples_2d = np.asarray(generated_samples_original)
    samples_2d = samples_2d.reshape(samples_2d.shape[0], samples_2d.shape[1])
    print(f'neves sample:{np.asarray(samples_2d).shape}')
    sample_predictions = model.predict(np.asarray(samples_2d))
    if model_type == 'proba':
        y_labels = np.argmax(sample_predictions, axis=1)
    else:
        y_labels = sample_predictions

    distances = np.asarray([np.linalg.norm(np.ones(len(cp_indexes))-x) for x in generated_samples_interpretable])
    weights = np.exp(-(np.abs(distances**2/(len(segment_indexes)**2))).reshape(n,))
    clf = Ridge()
    clf.fit(generated_samples_interpretable,y_labels,weights)
    return clf.coef_, cp_indexes





In [63]:
explanations = {}
i = 0 
noisy_explanations = {} # For Robustness later
new_dict = {'Coffee': datasets['Coffee']}
for data_idx in new_dict.keys():
    print('processing explanations for: ' + str(data_idx) + '\n')
    explanations[data_idx] = {}
    noisy_explanations[data_idx] = {}
    j = 0
    for model_idx in trained_models[data_idx].keys():
        print('processing explanations for: ' + str(model_idx) + '\n')
        explanations[data_idx][model_idx] = {}
        noisy_explanations[data_idx][model_idx] = {}
        explanation_set = datasets[data_idx][2][0:2]
        explanations[data_idx][model_idx]['LS'] = [LIMESegment(reshaper(x,j), explanation_set, trained_models[data_idx][model_idx], model_type=MODEL_TYPES[j], window_size=WINDOW_SIZES[i], cp=CPS[i]) for x in explanation_set]
        explanations[data_idx][model_idx]['N'] = [NEVES(x, trained_models[data_idx][model_idx], datasets[data_idx][0], model_type=MODEL_TYPES[j]) for x in explanation_set]
        explanations[data_idx][model_idx]['LF'] = [LEFTIST(reshaper(x,j), trained_models[data_idx][model_idx], datasets[data_idx][0], model_type=MODEL_TYPES[j],) for x in explanation_set]
        
        noisy_set = np.asarray([add_noise(x) for x in explanation_set])
        
        noisy_explanations[data_idx][model_idx]['LS'] = [LIMESegment(reshaper(x,j), noisy_set, trained_models[data_idx][model_idx], model_type=MODEL_TYPES[j], window_size=WINDOW_SIZES[i], cp=CPS[i]) for x in noisy_set]
        noisy_explanations[data_idx][model_idx]['N'] = [NEVES(x, trained_models[data_idx][model_idx], datasets[data_idx][0], model_type=MODEL_TYPES[j]) for x in noisy_set]
        noisy_explanations[data_idx][model_idx]['LF'] = [LEFTIST(reshaper(x,j), trained_models[data_idx][model_idx], datasets[data_idx][0], model_type=MODEL_TYPES[j],) for x in noisy_set]
        
        j = j + 1
    i = i + 1

processing explanations for: Coffee

processing explanations for: knn

(286,)
limesegment sample:(100, 286)
(286,)
limesegment sample:(100, 286)
(286, 1)
neves sample:(100, 286)
(286, 1)
neves sample:(100, 286)
leftist sample:(100, 286)
leftist sample:(100, 286)
(286,)
limesegment sample:(100, 286)
(286,)
limesegment sample:(100, 286)
(286,)
neves sample:(100, 286)
(286,)
neves sample:(100, 286)
leftist sample:(100, 286)
leftist sample:(100, 286)
processing explanations for: cnn

(286,)
limesegment sample:(100, 286)
4/4 [==============================] - 0s 3ms/step
(286,)
limesegment sample:(100, 286)
4/4 [==============================] - 0s 3ms/step
(286, 1)
neves sample:(100, 286)
4/4 [==============================] - 0s 4ms/step
(286, 1)
neves sample:(100, 286)
4/4 [==============================] - 0s 4ms/step
leftist sample:(100, 286)
4/4 [==============================] - 0s 3ms/step
leftist sample:(100, 286)
4/4 [==============================] - 0s 3ms/step
(286,)
limesegmen

In [64]:
evaluation_metrics = {}
for data_idx in datasets.keys():
    evaluation_metrics[data_idx] = {}
    j = 0 
    for model_idx in trained_models[data_idx].keys():
        evaluation_metrics[data_idx][model_idx] = {}
        for explanation_idx in explanations[data_idx][model_idx].keys():
            evaluation_metrics[data_idx][model_idx][explanation_idx] = {}
            # Robustness
            evaluation_metrics[data_idx][model_idx][explanation_idx]['Robustness'] = robustnessy(explanations[data_idx][model_idx][explanation_idx],
                                                                                         noisy_explanations[data_idx][model_idx][explanation_idx])
            explanation_set = datasets[data_idx][2][0:2]
            explanation_labels = datasets[data_idx][3][0:2]
            if j == 0:
                explanation_predictions = trained_models[data_idx][model_idx].predict(explanation_set.reshape(explanation_set.shape[:2]))
            
            else:
                explanation_predictions = trained_models[data_idx][model_idx].predict(explanation_set)
                
                # Faithfulness
            evaluation_metrics[data_idx][model_idx][explanation_idx]['Faithfulness'] = faithfulnessy(explanations[data_idx][model_idx][explanation_idx],explanation_set,explanation_labels,explanation_predictions,trained_models[data_idx][model_idx],model_type=MODEL_TYPES[j])
        j+=1
    

NameError: name 'robustnessy' is not defined